# ================================================
# PIPELINE DE MACHINE LEARNING COM PLN E VECTORDB
# ================================================

## INSTALAÇÃO DE DEPENDÊNCIAS

In [ ]:
# !pip install sentence-transformers
# !pip install chromadb
# !pip install pdfplumber
# !pip install nltk
# !pip install spacy
# !python -m spacy download pt_core_news_sm

## IMPORTS

In [22]:
import json
import csv
import re
import numpy as np
import pickle
import os
import spacy
import nltk
import chromadb
from nltk.corpus import stopwords
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import CharacterTextSplitter
from langchain_text_splitters import MarkdownHeaderTextSplitter

#nltk.download('stopwords') # rodar apenas uma vez

## ETAPAS

### 1. CARREGAMENTO DE ARQUIVOS

#### Definição das funções

In [5]:
def carregar_pdf(caminho_arquivo):
    """Carrega e extrai texto de um arquivo PDF"""
    try:
        import pdfplumber
        texto_completo = []
        with pdfplumber.open(caminho_arquivo) as pdf:
            for pagina in pdf.pages:
                texto = pagina.extract_text()
                if texto:
                    texto_completo.append(texto)
        return "\n".join(texto_completo)
    except ImportError:
        print("⚠ pdfplumber não disponível. Instale com: pip install pdfplumber")
        return ""

def carregar_csv(caminho_arquivo):
    """Carrega e extrai texto de um arquivo CSV"""
    textos = []
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for linha in reader:
            texto = " ".join([str(v) for v in linha.values()])
            textos.append(texto)
    return "\n".join(textos)

def carregar_json(caminho_arquivo):
    """Carrega e extrai texto de um arquivo JSON"""
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        dados = json.load(f)

    textos = []

    def extrair_texto_recursivo(obj):
        if isinstance(obj, dict):
            for valor in obj.values():
                extrair_texto_recursivo(valor)
        elif isinstance(obj, list):
            for item in obj:
                extrair_texto_recursivo(item)
        elif isinstance(obj, str):
            textos.append(obj)

    extrair_texto_recursivo(dados)
    return "\n".join(textos)

def carregar_txt(caminho_arquivo):
    """Carrega e extrai texto de um arquivo TXT"""
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        return f.read()

def carregar_arquivo(caminho_arquivo):
    """Função universal para carregar arquivos PDF, CSV ou JSON"""
    extensao = caminho_arquivo.split('.')[-1].lower()

    if extensao == 'pdf':
        return carregar_pdf(caminho_arquivo)
    elif extensao == 'csv':
        return carregar_csv(caminho_arquivo)
    elif extensao == 'json':
        return carregar_json(caminho_arquivo)
    elif extensao == 'txt':
        return carregar_txt(caminho_arquivo)
    else:
        raise ValueError(f"Formato não suportado: {extensao}")

#### Execução

In [6]:
texto = ""
texto = carregar_arquivo('introducaoml.pdf')
print(texto)

Título: Introdução ao Machine Learning
Machine learning é um campo da inteligência artificial que desenvolve algoritmos
capazes de aprender padrões a partir de dados. Os principais tipos incluem
aprendizado supervisionado, aprendizado não supervisionado e aprendizado por
reforço.
Aprendizado supervisionado envolve treinar um modelo com dados rotulados,
como prever preços de casas com base em características como tamanho,
localização e número de quartos. Aprendizado não supervisionado detecta
padrões ocultos em dados não rotulados, como segmentação de clientes em
marketing. Aprendizado por reforço ensina agentes a tomar decisões em
ambientes dinâmicos para maximizar recompensas.
Redes neurais profundas são usadas em visão computacional e processamento
de linguagem natural. Modelos como CNNs e Transformers têm revolucionado
tarefas como tradução automática, reconhecimento de imagens e geração de
texto.


### 2. PROCESSAMENTO DE LINGUAGEM NATURAL (PLN)

#### Definição das Stop Words

In [7]:
nlp = spacy.load('pt_core_news_sm')

API_STOPWORDS_PT = set(stopwords.words('portuguese'))
API_STOPWORDS_PT.remove('não')  # Mantendo a palavra "não"

STOPWORDS_EN = {'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 
                'has', 'he', 'in', 'is', 'it', 'its', 'of', 'on', 'that', 'the', 
                'to', 'was', 'will', 'with', 'this', 'but', 'they', 'have', 'had'}

# Carregando apenas as stopwords em português

STOPWORDS_ALL = API_STOPWORDS_PT

#### Pré-Processamento de Texto

In [8]:
def limpar_texto(texto):
    """Realiza limpeza básica do texto"""
    texto = texto.lower()
    texto = re.sub(r'\n\n', ' \n\n', texto) # manter quebras de parágrafo em caso de recursive chunking
    texto = re.sub(r'\n', ' \n', texto)
    texto = re.sub(r'\.', ' .', texto)
    texto = re.sub(r'[^a-z0-9áàâãéêíóôõúüçñ\s\n.]', ' ', texto) # ^ indica negação
    texto = re.sub(r'  +', ' ', texto) # multiplos espaços
    return texto.strip()

# Substituir acentos em caso de stemização

def tokenizar_simples(texto):
    """Tokeniza o texto (divisão por espaços)"""
    return texto.split()

def tokenizar_spacy(texto):
    """Tokeniza o texto usando spaCy"""
    doc = nlp(texto)
    return [token for token in doc]

def remover_stopwords(tokens, stopwords=STOPWORDS_ALL):
    """Remove stopwords em português e inglês"""
    return [token for token in tokens if token.text not in stopwords]

def lematizar_texto(tokens):
    """Pega a palavra lema de cada token no texto"""
    return [token.lemma_ for token in tokens]

def processar_texto_pln(texto):
    """Pipeline completo de PLN"""
    texto_limpo = limpar_texto(texto)
    palavras = tokenizar_spacy(texto_limpo)
    palavras_sem_stopwords = remover_stopwords(palavras)
    palavras_lemma = lematizar_texto(palavras_sem_stopwords)
    return ' '.join(palavras_lemma)

#### Execução

In [9]:
texto_processado = processar_texto_pln(texto)
print(texto_processado)

título introdução machine learning 
 machine learning campo inteligência artificial desenvolver algorit 
 capaz aprender padrão partir dado . principal tipo incluir 
 aprendizar supervisionar aprendizar não supervisionar aprendizar 
 reforço . 
 aprendizar supervisionar envolver treinar modelo dado rotular 
 prever preço casa base característica tamanho 
 localização número quarto . aprendizar não supervisionar detectar 
 padrão oculto dado não rotular segmentação cliente 
 marketing . aprendizar reforço ensina agente tomar decisão 
 ambiente dinâmico maximizar recompensa . 
 rede neural profundo usar visão computacional processamento 
 linguagem natural . modelo cnns transformer ter revolucionar 
 tarefa tradução automático reconhecimento imagem geração 
 texto .


### 3. CHUNKING COM OVERLAP

#### Tipos relevantes de chunking

##### Fixed-Length Chunking

Divide o texto em pedaços com número fixo de caracteres ou tokens. É simples de implementar e previsível, porém pode cortar informações importantes.

In [111]:
# EXEMPLO:

fixed_splitter = CharacterTextSplitter(
    separator=".",
    chunk_size=150,
    chunk_overlap=20
)

chunks = fixed_splitter.split_text(texto_processado)
print("n_chunks: ", len(chunks))
print(chunks)

n_chunks:  7
['título introdução machine learning \n machine learning campo inteligência artificial desenvolver algorit \n capaz aprender padrão partir dado', 'principal tipo incluir \n aprendizar supervisionar aprendizar não supervisionar aprendizar \n reforço', 'aprendizar supervisionar envolver treinar modelo dado rotular \n prever preço casa base característica tamanho \n localização número quarto', 'aprendizar não supervisionar detectar \n padrão oculto dado não rotular segmentação cliente \n marketing', 'aprendizar reforço ensina agente tomar decisão \n ambiente dinâmico maximizar recompensa', 'rede neural profundo usar visão computacional processamento \n linguagem natural', 'modelo cnns transformer ter revolucionar \n tarefa tradução automático reconhecimento imagem geração \n texto']


##### Recursive Character Chunking

Divide o texto progressivamente, mantendo mantendo estruturas naturais (parágrafos, frases), a melhor abordagem na maioria dos casos, superior ao fixed-length

In [29]:
# EXEMPLO:

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=20,
    separators=["\n\n", ".", "\n", " ", ""] # '.' tem mais valor semântico que '\n' pelo modo como o arquivo(pdf) é carregado
)

chunks = recursive_splitter.split_text(texto_processado)

for index, chunk in enumerate(chunks):
    chunks[index] = re.sub(r' \n', '', chunk)
    chunks[index] = re.sub(r' \.', '', chunks[index])
    #chunk = re.sub(r'\,', '', chunk)

print("n_chunks: ", len(chunks))
print(chunks)

n_chunks:  7
['título introdução machine learning machine learning campo inteligência artificial desenvolver algorit capaz aprender padrão partir dado', '. principal tipo incluir aprendizar supervisionar aprendizar não supervisionar aprendizar reforço', '. aprendizar supervisionar envolver treinar modelo dado rotular prever preço casa base característica tamanho localização número quarto', '. aprendizar não supervisionar detectar padrão oculto dado não rotular segmentação cliente marketing', '. aprendizar reforço ensina agente tomar decisão ambiente dinâmico maximizar recompensa', '. rede neural profundo usar visão computacional processamento linguagem natural', '. modelo cnns transformer ter revolucionar tarefa tradução automático reconhecimento imagem geração texto']


##### Document-Based Chunking

Divide o texto respeitando a estrutura hierárquica do documento, ótima abordagem para casos envolvendo textos com markdown, artigos científicos e acadêmicos.

In [ ]:
# EXEMPLO COM MARKDOWN:

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

chunks_document = markdown_splitter.split_text(documento_markdown)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=20
)

final_chunks = text_splitter.split_documents(chunks_document)

# chunks_document
# metadata definidos no headers_to_split_on -> chunk.metadata # {'Header 1': 'Introdução ao PLN'}
# conteúdo  # Texto ...

### 4. EMBEDDING

#### Definição da função

Modelo BERT é treinado especificamente para a Língua Portuguesa e é bom para tarefas de similaridade semântica

In [30]:
from sentence_transformers import SentenceTransformer

ENCODE_MODEL = 'rufimelo/bert-base-portuguese-cased-sts'
# ENCODE_MODEL = 'all-MiniLM-L6-v2' # leve, porém melhor para inglês

modelo = SentenceTransformer(ENCODE_MODEL)

def criar_embeddings(chunks):
    """Cria embeddings para os chunks"""
    embeddings = modelo.encode(chunks, show_progress_bar=True)
    return embeddings

Invalid model-index. Not loading eval results into CardData.


#### Execução

In [31]:
embedding = criar_embeddings(chunks)
print("Dimensão do embedding: ", embedding.shape)
print(embedding[:5])

Batches: 100%|██████████| 1/1 [00:09<00:00,  9.25s/it]

Dimensão do embedding:  (7, 1024)
[[-0.24512105 -0.2638833  -1.2534713  ...  0.75989485 -1.2004327
  -0.34133682]
 [ 0.68989885  0.05904028 -1.1700327  ...  0.56694835 -0.2709181
   0.9166821 ]
 [ 0.25678015 -0.3784146  -0.59672904 ...  0.5800669  -0.44971
   0.29745457]
 [-0.08029696 -0.29637185 -0.79932034 ...  1.2091796  -0.7239852
  -0.0411283 ]
 [ 0.15125325  0.20062709 -0.46873796 ...  0.02203956 -1.8070877
   0.04457791]]


### 5. VECTOR DATABASE

#### Inicializando cliente

In [34]:
client = chromadb.PersistentClient(path='./chroma_db')

collection = client.get_or_create_collection(name='documentos_ml', embedding_function=None)

print(f"✓ ChromaDB inicializado: {collection.count()} documentos na coleção")

✓ ChromaDB inicializado: 7 documentos na coleção


#### Adicionar ao chroma

In [ ]:
# Criar ids únicos
ids = [f"chunk_{i}" for i in range(len(chunks))]

# collection.add(
#     ids=ids,
#     documents=chunks,
#     embeddings=embedding.tolist()  # converter para lista
# )

✓ 7 chunks adicionados ao ChromaDB
Total de documentos na coleção: 7


### 6. BUSCA VETORIAL (QUERY)

In [ ]:
questions = ["O que é aprendizado de máquina?", "Quais são os tipos de aprendizado de máquina?"]

for question in questions:
    question_embedding = modelo.encode([question])
    resultados = collection.query(
        query_embeddings=question_embedding.tolist(),
        n_results=3
    )
    print(f"\nPergunta: {question}")
    for doc, score in zip(resultados['documents'][0], resultados['distances'][0]):
        print(f" - Documento: {doc} (Score: {score})")


Pergunta: O que é aprendizado de máquina?
 - Documento: título introdução machine learning machine learning campo inteligência artificial desenvolver algorit capaz aprender padrão partir dado (Score: 328.4739685058594)
 - Documento: . rede neural profundo usar visão computacional processamento linguagem natural (Score: 438.04241943359375)
 - Documento: . modelo cnns transformer ter revolucionar tarefa tradução automático reconhecimento imagem geração texto (Score: 483.20989990234375)

Pergunta: Quais são os tipos de aprendizado de máquina?
 - Documento: título introdução machine learning machine learning campo inteligência artificial desenvolver algorit capaz aprender padrão partir dado (Score: 359.50006103515625)
 - Documento: . rede neural profundo usar visão computacional processamento linguagem natural (Score: 453.0020446777344)
 - Documento: . modelo cnns transformer ter revolucionar tarefa tradução automático reconhecimento imagem geração texto (Score: 478.9754333496094)
